# YOLO v3 Object Detection

Let's see how to use the state of the art in object detection! Please make sure to watch the video, there is no code along here, since we can't reasonably train the YOLOv3 network ourself, instead we will use a pre-established version.

CODE SOURCE: https://github.com/xiaochus/YOLOv3

REFERENCE (for original YOLOv3): 

        @article{YOLOv3,  
              title={YOLOv3: An Incremental Improvement},  
              author={J Redmon, A Farhadi },
              year={2018} 
--------
----------
## YOU MUST WATCH THE VIDEO LECTURE TO PROPERLY SET UP THE MODEL AND WEIGHTS. THIS NOTEBOOK WON'T WORK UNLESS YOU FOLLOW THE EXACT SET UP SHOWN IN THE VIDEO LECTURE.
-------
-------

In [1]:
import os
import time
import cv2
import numpy as np
from model.yolo_model import YOLO

In [2]:
def process_image(img):
    """Resize, reduce and expand image.

    # Argument:
        img: original image.

    # Returns
        image: ndarray(64, 64, 3), processed image.
    """
    image = cv2.resize(img, (416, 416),
                       interpolation=cv2.INTER_CUBIC)
    image = np.array(image, dtype='float32')
    image /= 255.
    image = np.expand_dims(image, axis=0)

    return image

In [3]:
def get_classes(file):
    """Get classes name.

    # Argument:
        file: classes name for database.

    # Returns
        class_names: List, classes name.

    """
    with open(file) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]

    return class_names

In [4]:
def draw(image, boxes, scores, classes, all_classes):
    """Draw the boxes on the image.

    # Argument:
        image: original image.
        boxes: ndarray, boxes of objects.
        classes: ndarray, classes of objects.
        scores: ndarray, scores of objects.
        all_classes: all classes name.
    """
    for box, score, cl in zip(boxes, scores, classes):
        x, y, w, h = box

        top = max(0, np.floor(x + 0.5).astype(int))
        left = max(0, np.floor(y + 0.5).astype(int))
        right = min(image.shape[1], np.floor(x + w + 0.5).astype(int))
        bottom = min(image.shape[0], np.floor(y + h + 0.5).astype(int))

        cv2.rectangle(image, (top, left), (right, bottom), (255, 0, 0), 2)
        cv2.putText(image, '{0} {1:.2f}'.format(all_classes[cl], score),
                    (top, left - 6),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, (0, 0, 255), 1,
                    cv2.LINE_AA)

        print('class: {0}, score: {1:.2f}'.format(all_classes[cl], score))
        print('box coordinate x,y,w,h: {0}'.format(box))

    print()

In [5]:
def detect_image(image, yolo, all_classes):
    """Use yolo v3 to detect images.

    # Argument:
        image: original image.
        yolo: YOLO, yolo model.
        all_classes: all classes name.

    # Returns:
        image: processed image.
    """
    pimage = process_image(image)

    start = time.time()
    boxes, classes, scores = yolo.predict(pimage, image.shape)
    end = time.time()

    print('time: {0:.2f}s'.format(end - start))

    if boxes is not None:
        draw(image, boxes, scores, classes, all_classes)

    return image

In [7]:
def detect_video(video, yolo, all_classes):
    """Use yolo v3 to detect video.

    # Argument:
        video: video file.
        yolo: YOLO, yolo model.
        all_classes: all classes name.
    """
    video_path = os.path.join("videos", "test", video)
    camera = cv2.VideoCapture(video_path)
    cv2.namedWindow("detection", cv2.WINDOW_AUTOSIZE)

    # Prepare for saving the detected video
    sz = (int(camera.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    fourcc = cv2.VideoWriter_fourcc(*'mpeg')

    
    vout = cv2.VideoWriter()
    vout.open(os.path.join("videos", "res", video), fourcc, 20, sz, True)

    while True:
        res, frame = camera.read()

        if not res:
            break

        image = detect_image(frame, yolo, all_classes)
        cv2.imshow("detection", image)

        # Save the video frame by frame
        vout.write(image)

        if cv2.waitKey(110) & 0xff == 27:
                break

    vout.release()
    camera.release()
    

In [8]:
yolo = YOLO(0.6, 0.5)
file = 'data/coco_classes.txt'
all_classes = get_classes(file)

### Detecting Images

In [13]:
f = 'manyPeople.jpg'
path = 'images/'+f
image = cv2.imread(path)
image = detect_image(image, yolo, all_classes)
cv2.imwrite('images/res/' + f, image)

time: 0.85s
class: person, score: 0.99
box coordinate x,y,w,h: [2704.85779965 2372.45407712  882.79332104 2168.12946966]
class: person, score: 0.99
box coordinate x,y,w,h: [1404.48552263 2433.64632797  774.01531258 2123.67594534]
class: person, score: 0.98
box coordinate x,y,w,h: [ 701.10133196 2354.35781264  861.45341077 2146.43555671]
class: person, score: 0.97
box coordinate x,y,w,h: [ -18.10657351 2357.61092448  811.08013287 2167.1918487 ]
class: person, score: 0.85
box coordinate x,y,w,h: [2011.69662952 2382.60456061 1111.20465571 2116.32103568]



True

# Detecting on Video

In [14]:
# # detect videos one at a time in videos/test folder    
video = 'Pehla Pyaar Amul Pyaar.mp4'
detect_video(video, yolo, all_classes)

time: 0.80s
time: 0.83s
time: 0.84s
time: 0.86s
time: 0.86s
time: 0.82s
time: 0.76s
time: 0.68s
time: 0.67s
class: bottle, score: 0.60
box coordinate x,y,w,h: [287.95650482   9.93973553  84.63904381 155.10908961]

time: 0.65s
class: bottle, score: 0.86
box coordinate x,y,w,h: [288.09539795  10.17366707  84.52417374 159.59425807]

time: 0.67s
time: 0.65s
time: 0.66s
time: 0.66s
time: 0.69s
time: 0.94s
time: 0.69s
time: 0.71s
class: bottle, score: 0.77
box coordinate x,y,w,h: [223.27054977   6.2607801  122.57369041 242.97223806]

time: 0.66s
class: bottle, score: 0.83
box coordinate x,y,w,h: [289.12620544  29.88771558  79.61510181 232.71366835]
class: bottle, score: 0.68
box coordinate x,y,w,h: [217.23955154  11.75806403 121.49703979 268.26465368]

time: 0.66s
class: bottle, score: 0.81
box coordinate x,y,w,h: [288.55365753  51.49752259  73.9234972  215.9909749 ]

time: 0.65s
class: bottle, score: 0.85
box coordinate x,y,w,h: [288.74866486  52.73295879  74.09981728 214.19329405]

time: 0

time: 0.76s
class: bottle, score: 0.94
box coordinate x,y,w,h: [339.10888672  65.33076882  82.77020454 245.41858435]
class: bottle, score: 0.88
box coordinate x,y,w,h: [260.84127426  70.86203098  83.86979103 233.80884647]
class: bottle, score: 0.86
box coordinate x,y,w,h: [194.45644379  47.94668555  74.96737957 277.55827188]
class: bottle, score: 0.67
box coordinate x,y,w,h: [428.30474854  59.65346575  95.11049271 255.12712955]

time: 0.77s
class: bottle, score: 0.96
box coordinate x,y,w,h: [338.92410278  66.37199163  84.15334702 241.4303112 ]
class: bottle, score: 0.85
box coordinate x,y,w,h: [193.37581635  39.49543118  77.07306862 293.44188452]
class: bottle, score: 0.78
box coordinate x,y,w,h: [261.2733078   67.60500312  83.87465477 241.19374037]
class: bottle, score: 0.72
box coordinate x,y,w,h: [427.82833099  63.45449924  95.54171562 248.11373234]

time: 0.75s
class: bottle, score: 0.93
box coordinate x,y,w,h: [340.7623291   65.07753611  81.30898476 244.02493   ]
class: bottle, sc

time: 0.69s
class: bottle, score: 0.97
box coordinate x,y,w,h: [191.34822845  60.21781325  76.94038868 224.69817638]
class: bottle, score: 0.94
box coordinate x,y,w,h: [339.13463593  48.44610214  76.85226917 250.766716  ]
class: bottle, score: 0.78
box coordinate x,y,w,h: [265.99115372  60.75792432  72.72753239 224.2957592 ]
class: bottle, score: 0.68
box coordinate x,y,w,h: [407.54249573  69.28314328  86.09377861 207.79038906]

time: 0.70s
class: bottle, score: 0.96
box coordinate x,y,w,h: [192.03670502  59.99411702  76.15087509 224.5557189 ]
class: bottle, score: 0.95
box coordinate x,y,w,h: [339.21661377  46.97816133  76.76009655 253.19295645]
class: bottle, score: 0.78
box coordinate x,y,w,h: [266.40583038  59.75595832  71.95065498 225.93336582]
class: bottle, score: 0.74
box coordinate x,y,w,h: [407.50953674  70.1429522   85.91835976 205.90074062]

time: 0.70s
class: bottle, score: 0.89
box coordinate x,y,w,h: [339.15653229  49.34901953  78.94048214 249.02070522]
class: bottle, sc

time: 0.72s
class: person, score: 0.96
box coordinate x,y,w,h: [215.47119141 121.15240932 183.50439072 242.06187487]

time: 0.70s
class: person, score: 0.97
box coordinate x,y,w,h: [215.13816833 114.89453673 193.65167618 252.57132769]

time: 0.74s
class: person, score: 0.96
box coordinate x,y,w,h: [220.9331131  117.2042191  192.90147781 245.87872267]

time: 0.71s
class: person, score: 0.96
box coordinate x,y,w,h: [251.06246948 118.72269273 165.10887146 241.64203405]

time: 0.71s
class: person, score: 0.95
box coordinate x,y,w,h: [270.1531601  114.61422443 157.37265587 237.44643688]

time: 0.69s
class: person, score: 0.96
box coordinate x,y,w,h: [269.67754364 115.04486918 158.74952316 236.46682978]

time: 0.68s
class: person, score: 0.99
box coordinate x,y,w,h: [292.34386444 109.92716074 144.249897   246.35102749]

time: 0.72s
class: person, score: 0.99
box coordinate x,y,w,h: [294.04319763 111.82481289 151.04766846 249.32171345]

time: 0.70s
class: person, score: 0.99
box coordinate x,

time: 0.70s
class: person, score: 0.99
box coordinate x,y,w,h: [ 68.52308273  79.25222754 274.12853241 276.56280756]
class: person, score: 0.69
box coordinate x,y,w,h: [362.49599457  61.15563154  75.64033031 109.08005476]
class: bicycle, score: 1.00
box coordinate x,y,w,h: [322.38456726 112.01281428 154.6582222   76.3129288 ]

time: 0.70s
class: person, score: 0.99
box coordinate x,y,w,h: [ 68.26205254  81.36796474 279.23936844 272.74108887]
class: bicycle, score: 1.00
box coordinate x,y,w,h: [325.91255188 111.11646295 157.12519646  76.43054903]

time: 0.70s
class: person, score: 0.99
box coordinate x,y,w,h: [ 68.87711525  80.40845275 276.36789322 273.8499999 ]
class: bicycle, score: 1.00
box coordinate x,y,w,h: [325.76847076 110.83846807 157.76403427  76.90094948]

time: 0.70s
class: person, score: 0.99
box coordinate x,y,w,h: [ 61.77219391  83.71071339 290.37937164 267.75835991]
class: bicycle, score: 1.00
box coordinate x,y,w,h: [336.33712769 107.51613379 157.42699623  80.16486526]


...

time: 0.70s
class: person, score: 0.99
box coordinate x,y,w,h: [115.25966644  49.29719925 327.2694397  280.95487118]

time: 0.70s
class: person, score: 0.99
box coordinate x,y,w,h: [113.34770203  49.47727203 329.7102356  281.44294739]

time: 0.69s
class: person, score: 0.99
box coordinate x,y,w,h: [114.19332504  48.90184164 329.19059753 281.95042133]

time: 0.68s
class: person, score: 1.00
box coordinate x,y,w,h: [114.38663483  47.52956986 330.206604   284.42219496]

time: 0.71s
class: person, score: 1.00
box coordinate x,y,w,h: [115.42976379  48.46381545 327.8597641  282.70549536]

time: 0.71s
class: person, score: 0.99
box coordinate x,y,w,h: [112.47343063  48.7728703  331.37760162 282.79514551]

time: 0.71s
class: person, score: 0.99
box coordinate x,y,w,h: [114.08296585  49.88004327 330.18169403 280.82048178]

time: 0.70s
class: person, score: 0.99
box coordinate x,y,w,h: [117.11194992  50.62824011 327.11395264 279.96026516]

time: 0.69s
class: person, score: 0.99
box coordinate x,

time: 0.69s
class: person, score: 0.98
box coordinate x,y,w,h: [278.8230896   20.5680263  299.96650696 332.79074907]

time: 0.69s
class: person, score: 0.98
box coordinate x,y,w,h: [279.00875092  23.6723721  307.42963791 327.04962015]

time: 0.70s
class: person, score: 0.98
box coordinate x,y,w,h: [302.16686249  23.56680036 315.36071777 330.39963484]

time: 0.70s
class: person, score: 0.98
box coordinate x,y,w,h: [306.03347778  20.7914114  306.04856491 336.58113956]

time: 0.69s
class: person, score: 0.98
box coordinate x,y,w,h: [304.89786148  22.11846113 309.1942215  334.26723003]

time: 0.69s
class: person, score: 0.98
box coordinate x,y,w,h: [311.35210037  21.2512815  303.43486786 337.42788076]
class: person, score: 0.80
box coordinate x,y,w,h: [  2.67558098   7.84927011 254.05435562 338.6272788 ]

time: 0.71s
class: person, score: 1.00
box coordinate x,y,w,h: [402.31700897  64.8741281  208.42054367 294.29547071]
class: person, score: 0.96
box coordinate x,y,w,h: [ 20.06565571  81.9

time: 0.71s
class: person, score: 0.93
box coordinate x,y,w,h: [458.75617981  71.32325292 175.62076569 217.56596804]
class: person, score: 0.83
box coordinate x,y,w,h: [ 11.697402    89.36862946 131.25136375 249.36758995]

time: 0.70s
class: person, score: 0.96
box coordinate x,y,w,h: [458.56391907  72.23059058 176.52462006 214.31723356]
class: person, score: 0.88
box coordinate x,y,w,h: [  7.93869495  89.40266132 135.10497093 274.93226051]

time: 0.70s
class: person, score: 0.96
box coordinate x,y,w,h: [458.69216919  67.35532165 177.87530899 226.16525888]
class: person, score: 0.90
box coordinate x,y,w,h: [  9.6042347   89.71375465 131.33057594 275.99033833]

time: 0.69s
class: person, score: 0.97
box coordinate x,y,w,h: [459.33677673  68.58323693 176.88512802 223.03737402]
class: person, score: 0.88
box coordinate x,y,w,h: [  9.8293829   90.40881157 132.32055664 274.6231842 ]

time: 0.69s
class: person, score: 0.98
box coordinate x,y,w,h: [459.67857361  68.89422297 175.90354919 221.1

time: 0.70s
class: person, score: 1.00
box coordinate x,y,w,h: [ 67.90349007  96.39155388 134.88981247 260.27049065]
class: person, score: 0.90
box coordinate x,y,w,h: [434.76387024 124.96512651 148.27228546 227.7523756 ]

time: 0.69s
class: person, score: 1.00
box coordinate x,y,w,h: [ 69.52156067  92.39287376 137.0281601  265.76202393]
class: person, score: 0.89
box coordinate x,y,w,h: [435.50094604 128.00122619 147.69130707 224.25754309]

time: 0.70s
class: person, score: 1.00
box coordinate x,y,w,h: [ 69.08111572  86.55958414 139.74124908 275.58564663]
class: person, score: 0.92
box coordinate x,y,w,h: [435.26374817 129.78517413 144.91592407 221.61863565]

time: 0.72s
class: person, score: 1.00
box coordinate x,y,w,h: [ 69.78236675  87.90868163 138.5258007  272.7709794 ]
class: person, score: 0.92
box coordinate x,y,w,h: [437.21237183 130.09498    141.65967941 221.50336504]

time: 0.70s
class: person, score: 1.00
box coordinate x,y,w,h: [ 67.61677742  88.75711799 141.72502518 271.6

time: 0.69s
class: bottle, score: 0.96
box coordinate x,y,w,h: [213.43309402  85.95067978  78.45403194 174.6662879 ]

time: 0.71s
class: bottle, score: 0.96
box coordinate x,y,w,h: [213.32176208  84.70920861  78.20917606 176.78387046]

time: 0.70s
class: bottle, score: 0.96
box coordinate x,y,w,h: [213.44388962  85.089342    78.29268456 175.53119302]

time: 0.70s
class: bottle, score: 0.95
box coordinate x,y,w,h: [214.03804779  86.45365834  77.63987064 172.15071917]

time: 0.70s
class: bottle, score: 0.93
box coordinate x,y,w,h: [213.53767395  86.8135196   78.96321774 170.28082252]

time: 0.70s
class: bottle, score: 0.96
box coordinate x,y,w,h: [214.68408585  82.13663757  77.66489506 178.48297477]

time: 0.69s
class: bottle, score: 0.92
box coordinate x,y,w,h: [215.49100876  57.77745366  77.40536213 204.17987823]

time: 0.70s
class: bottle, score: 0.94
box coordinate x,y,w,h: [214.42623138  55.70670247  78.68349552 207.35215902]

time: 0.70s
class: bottle, score: 0.94
box coordinate x,